In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from threading import Thread

class StreamingThread(Thread):
    def __init__(self, ssc):
        Thread.__init__(self)
        self.ssc = ssc
    def run(self):
        ssc.start()
        ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
spark

In [5]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType
 

In [6]:
#get and store the data

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    rdd.coalesce(1).saveAsTextFile('file:///C:/Users/lenne/Desktop/spark/coding_and_data/data/new_data')
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()

In [7]:
ssc = StreamingContext(sc, 5000)

In [8]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [9]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

========= 2021-05-11 11:23:20 =========
+----------+-------------------+--------------------+
|     label|           tweet_id|          tweet_text|
+----------+-------------------+--------------------+
|    #china|1392042685905649665|Watch: New batch ...|
|    #china|1392042664007290881|Helicopter explod...|
|    #china|1392042599733547008|#███████'s #█████...|
|#inflation|1392043004152668161|Perspective matte...|
|#inflation|1392043175762534406|76% analysts at P...|
|#inflation|1392043115515506691|𝗗𝗔𝗜𝗟𝗬 𝗕𝗨𝗟...|
|#inflation|1392043363604439041|China's domestic ...|
|#inflation|1392043891310821376|50% analysts at P...|
|    #china|1392044122169434114|The State Council...|
|    #china|1392044077139513344|Congratulations ?...|
|    #china|1392043931827937284|@globaltimesnews ...|
|    #china|1392043914127831047|Chinese Foundry w...|
|#inflation|1392044241132654595|I saw this lumber...|
|#inflation|1392044166830432257|76% analysts at P...|
|    #china|1392044325580664833|Everything has a .

In [ ]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
